# Module 9. Natural language processing
# Topic modeling

## Lecture objectives
1. Provide practice with importing PDFs and handling exceptions
2. Demonstrate how to estimate a topic model using LDA
3. Discuss how to adjust the model parameters in the search for a meaningful topic model

So far, we've got some text into Python from a PDF, cleaned it up, and split the string into sentences and words. We've also done simple word counts. Now, we'll see what topic modeling - one method under the broader umbrella of natural language processing - can do.

Two examples in urban studies that use topic modeling are [Han et al.](https://www.tandfonline.com/doi/full/10.1080/01944363.2020.1831401) who analyze election communications, and [Brinkley & Stahmer](https://journals.sagepub.com/doi/abs/10.1177/0739456X21995890) who analyze General Plans. But let's say a little about the principles first.

Topic modeling is a method to identify patterns in text documents. It's an *unsupervised* machine learning technique, i.e. it doesn't need to be "trained" on a dataset of known topics. Rather, it identifies topics from the ground up. 

In that sense, topic modeling is related to cluster analysis. However, a document can belong (partially) to multiple topics, where an observation is typically assigned to a single cluster.

But both cluster analysis and topic modeling perform *dimensionality reduction*. Suppose we have 1000 distinct words in a document, that we reduce to 10 topics. We are going from a 1000-dimensional space to a 10-dimensional space. Obviously, 10 dimensions are much easier to interpret.

There are several algorithms that implement topic modeling; we'll focus on Latent Dirichlet Allocation (LDA) (as in the Han et al. paper). We'll use the `gensim` library, although `sklearn` is a popular alternative. `gensim` is dedicated to NLP, while `sklearn` is a more general machine-learning library.

We'll start with looking at a selection of Climate Action Plans (CAPs) from cities and counties in California. These are some of the plans I analyzed [in a project on equity in CAPS](https://journals.sagepub.com/doi/10.1177/0739456X211072527) with Hillary Angelo, Key MacFarlane, and James Sirigotis. Thanks to these collaborators for permission to share this dataset.

The plans are in a directory in your GitHub repository. We can see the list using `os`.

In [ ]:
import os
filelist = os.listdir('../data/CAPs')  

# Look at the first 10
filelist[:10]

Rather than writing code to read each file individually, let's write a function which takes a filename, and returns the text of that file. We've seen all this code before.

We can then create a list of documents. Each element of that list will be the text of a single CAP.

In [ ]:
from pdfminer.high_level import extract_text
import re

def readPDF(filename):
    txt = extract_text('../data/CAPs/'+filename)
    
    # use regex to remove punctuation, numbers, etc.
    txt = re.sub(r"[^A-z\s]", "", txt)
    # and to remove whitepace
    txt = re.sub(r"\s+", " ", txt) 
    
    print('Finished {}'.format(filename))
    return txt

# use a list comprehension to read in all the files
# only do those that end with .pdf
caps = [readPDF(fn) for fn in filelist if fn.endswith('.pdf')]

Let's look at some random parts of a couple of plans to make sure they loaded.

In [ ]:
print(caps[0][10000:10200])
print(caps[2][10000:10200])

Now, as before, let's remove the stop words. 

We'll use a list comprehension again, but this time in a nested way.  In the outer list, we loop over plans. In the inner list, we loop over words in that plan.

We could have done this another way through adding to the `readPDF` function, and making that return a list of words that exclude stop words.

At this point, we might want to lemmatize as well, but we'll skip that here.

In [ ]:
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize, sent_tokenize

swords = stopwords.words('english')
wordlists = [[word for word in word_tokenize(cap.lower()) 
                 if word not in swords] 
                 for cap in caps]

So for each plan, we have a list of words. For example, the first ten words from the first plan are as follows.

In [ ]:
wordlists[0][:10]

Which makes sense when we look at the start of that plan.

In [ ]:
caps[0][:100]

We are at the point where we can do topic modeling.

The `gensim` documentation is pretty thorough.

Note that there are lots of options. The most important are:
* `corpus`: the text. More on the format for this below
* `num_topics`: how many topics you want to identify
* `alpha`: the expected distribution of topics across documents (i.e., are topics concentrated in a few documents)
* `eta`:  (sometimes called `beta`): the expected distribution of words across topics (i.e., are words concentrated in a few topics) 

However, sensible defaults are provided. So normally, a good approach is to start with the defaults and adjust accordingly. 

In [ ]:
import gensim
help(gensim.models.LdaMulticore)

[The `gensim` documentation](https://radimrehurek.com/gensim/models/ldamulticore.html) provides some simple examples that help with getting up and running. There's also many other examples online. 

Conceptually, we need to:
* Convert our text to a list of lists. The outer list is each plan, and the inner list is words in that plan. We did this already.
* Create a gensim Dictionary (basically, each word gets an integer id)
* Feed that to the `gensim.models.LdaMulticore` function

The tricky part here is getting the data into the format (e.g. a list of words, list of word ids, string) that `gensim` expects. I did this mostly by adapting the examples on the `gensim` website.

Here, we choosing 5 topics - an arbitrary number. We aren't specifying the `alpha` and `eta` hyperparameters. Rather, we are using the `gensim` defaults.

In [ ]:
dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists]
# LdaMulticore uses multiple cores (thus, it runs faster)
# If you have problems, try replacing LdaMulticore with LdaModel
model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=5)

The `model` object now contains the results of our LDA model. We can explore some of its attributes and functions. The [documentation](https://radimrehurek.com/gensim/models/LdaModel.html#module-gensim.models.LdaModel) is also fairly comprehensive.

In [ ]:
model.show_topics()

In [ ]:
model.show_topic(0)

## Troubleshooting
So it looks like we have some topics. But there is a lot of nonsense. 

First of all, there are some one and two letter words.

Second, what is `cidcid`? 

Let's troubleshoot. We can find a plan that has this string in it.

In [ ]:
for cap in caps:
    if 'cidcid' in cap:
        # stop once we find that
        break
        
# look at the start of that plan
cap[:6000]

Who knows what caused that. PDFs are hard to parse!

So let's drop everything from our wordlist that is:
* Less than 3 letters
* Has `cid` in it

We can add these conditions. Note that we check separately to see if the word is equal to `cid`, or if multiple cids are in the word. This avoids dropping words like "incidental."

In [ ]:
swords = stopwords.words('english')
wordlists = [[word for word in word_tokenize(cap.lower()) 
                 if word not in swords and len(word)>2 and word!='cid' and 'cidcid' not in word] 
                 for cap in caps]

And estimate our topic model again.

In [ ]:
dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists]
model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=5)
model.show_topics()

This looks better! 

## Visualizing topic models
How can we visualize the topics?

The data format looks pretty straightforward. Each topic is a list of tuples of (word, weight). With some effort, we might be able to plot this ourselves. 

But a quick web search reveals that there is a [Python library specificially designed to visualize LDA outputs](https://pyldavis.readthedocs.io/en/latest/readme.html)! 

In [ ]:
import pyLDAvis
import pyLDAvis.gensim_models   # note that in previous versions this was called pyLDAvis.gensim
pyLDAvis.enable_notebook()
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

How do we interpret the visualization? You can find a detailed explanation [here](https://aclanthology.org/W14-3110.pdf). In short:
* The size of each circle is proportional to the frequency of that topic
* The distance between circles shows how closely they are related
* The bars represent the overall frequency of a word (blue) and the frequency within that topic
* The "relevance metric" slider controls the sorting. The default is to sort by overall frequency within a topic. As you slide to the left, words that are disproportionately frequent in that topic rise to the top.

## Making sense of topic models
So what does all this mean?

This is where exploratory analysis comes in. We probably want to adjust the number of topics and the parameters, until we find a set of topics that makes intuitive sense.

If we were doing regression analysis, this would be "fishing" (bad!). But in unsupervised machine learning, this type of exploration is an inherent part of the process. We aren't testing hypotheses, just searching for patterns and understanding.

Remember that the `LdaMulticore` function takes the `num_topics`, `alpha` and `eta` parameters. We could adjust those. We might also want to exclude certain uninformative words based on our present context — perhaps "city" or "climate."

`alpha` controls the expected distribution of topics across documents. A higher value of `alpha` means that each document is expected to contain more of a mix of topics, rather than focusing on a few topics.

`eta` (sometimes called beta) controls the expected distribution of words across topics. A higher value of `eta` means that topics are more similar in terms of their mixture of words.

For climate plans, let's assume that documents have more of a mix of topics, so we'll set `alpha=0.9`. For `eta`, it's unclear what to expect, but let's try `eta=0.5` for now.

In [ ]:
words_to_exclude = ['city','climate','action','emissions','ghg','reduction','plan']

wordlists = [[word for word in word_tokenize(cap.lower()) 
                 if word not in swords and word not in words_to_exclude
                     and len(word)>2 and word!='cid' and 'cidcid' not in word] 
                 for cap in caps]
dictionary = gensim.corpora.Dictionary(wordlists)
corpus = [dictionary.doc2bow(wl) for wl in wordlists]

model = gensim.models.LdaMulticore(corpus, id2word=dictionary, num_topics=5, alpha = 0.7, eta=0.5)
pyLDAvis.gensim_models.prepare(model, corpus, dictionary)

That gave some insights into what types of topics were included in plans, but it was hard to find a meaningful group of topics. We could experiment further with the hyperparameters. But it could also be a limitation of the small number of plans (30) in our sample. With a few hundred plans, we might generate further insights.

<div class="alert alert-block alert-info">
<strong>Exercise:</strong> Experiment with the parameters (and perhaps the list of words that you exclude), and see if you can come up with sensible topics.
</div>

<div class="alert alert-block alert-info">
<h3>Key Takeaways</h3>
<ul>
  <li>Topic modeling is a form of dimensionality reduction. The aim is to make your data easier to interpret.</li>
  <li>Exploration and iteration are the keys.</li>
  <li>After your first set of results, maybe there are texts or words that you want to exclude.</li>
  <li>Then, experiment with adjusting the parameters.</li>
    <li>Success is defined based on whether you find the results useful!</li>
</ul>
</div>